# Analysis across sessions

In [ ]:
import pandas as pd
import os
import json
import numpy as np
from itertools import groupby
import matplotlib.pyplot as plt
from scipy import stats,signal
import matplotlib as mpl
from sklearn.linear_model import LogisticRegression
import random
import re
import csv
from IPython.display import HTML, display, Image
import tabulate
import math as m
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
current_path = os.path.abspath(os.getcwd())
parent_path = os.path.abspath(os.path.join(current_path, os.pardir))
grand_parent_path = os.path.abspath(os.path.join(parent_path, os.pardir))
main_path = os.path.abspath(os.path.join(grand_parent_path, os.pardir))

path_results = main_path+'/results/gabor/'
path_plots = main_path+'/scripts/analysis/figures/'

In [ ]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, main_path+'/scr')
import my_functions as myf

In [ ]:
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['axes.titlesize'] = 18
mpl.rcParams['axes.labelsize'] = 18
mpl.rcParams['lines.markersize'] = 10
mpl.rcParams['xtick.labelsize'] = 20
mpl.rcParams['ytick.labelsize'] = 20
mpl.rcParams['axes.linewidth'] = 3
#mpl.rcParams['xtick.major.size'] = 20
mpl.rcParams['xtick.major.width'] = 4
#mpl.rcParams['xtick.minor.size'] = 10
mpl.rcParams['xtick.minor.width'] = 2
mpl.rcParams['ytick.major.width'] = 4
mpl.rcParams['ytick.minor.width'] = 2

fday = [1,2,3,4,5,6,7,8,9,10]
fsession = [1,2]

adf = pd.read_csv(path_results+'preanalyzed.csv')  

userids = adf['userID_x'].unique()
userids = sorted(userids)
nsub = len(userids)
sessionids = adf['sessionID_x'].unique()
sessionids = sorted(sessionids)

nsub = len(userids)

# morning df
mdf = adf[adf['sessionID_x']%2==1]
# evening df
edf = adf[adf['sessionID_x']%2==0]
edf_sin_nan = edf.copy()
edf_sin_nan['sleep']=list(mdf['sleep'])

key_SR = ['mood','food', 'real_stress','sleep']

In [ ]:
key_PV = ['no_sigma_x','no_bias_x','Dsigma_oo','HmiddleDO','Ssigma_oo','HmiddleSO']

In [ ]:
pair_SR_PV,pair_SR_PV2plot = [],[]
for pv in key_PV:
    aux = []
    for sr in key_SR:
        pair_SR_PV.append((sr,pv))
        aux.append((sr,pv))
    pair_SR_PV2plot.append(aux)

In [ ]:
corr_SR_PV, p_SR_PV, SIG_SR_PV, NOS_SR_PV = {},{},{},{}
for pair in pair_SR_PV:
    if 'sleep' in pair:  
        LR = [myf.Linear_Regr(np.array(mdf[mdf['userID_x']==part][pair[0]]),\
                            np.array(mdf[mdf['userID_x']==part][pair[1]])) for part in userids]
    else:
        LR = [myf.Linear_Regr(np.array(adf[adf['userID_x']==part][pair[0]]),\
                            np.array(adf[adf['userID_x']==part][pair[1]])) for part in userids]
    corr_SR_PV[pair] = [LR[k].r_value for k in range(len(userids))]
    p_SR_PV[pair] = stats.ttest_1samp(corr_SR_PV[pair],0)[1]
    
    LR_p_value = [LR[k].p_value for k in range(len(userids))]
    LR_p_value = np.array(LR_p_value)
    indSIG = np.where(LR_p_value<0.05)
    indNOS = np.where(LR_p_value>=0.05)
    SIG_SR_PV[pair] = [corr_SR_PV[pair][j] for j in indSIG[0]]
    NOS_SR_PV[pair] = [corr_SR_PV[pair][j] for j in indNOS[0]]

In [ ]:
round_p_PV = np.array([myf.roundP(p_SR_PV[(pair)]) for pair in pair_SR_PV])
round_p_PV = np.reshape(round_p_PV,(6,4))
p_value = [['p_value']+key_SR]
ind = -1
for key in key_PV:
    ind += 1
    p_value.append([key]+list(round_p_PV[ind]))

display(HTML(tabulate.tabulate(p_value, tablefmt='html')))

In [ ]:
fig, ax = plt.subplots(6,4,figsize=(22,24))
plt.subplots_adjust(hspace = 1)  

for ii in range(6):
    for jj in range(4):
        ax[ii,jj].hist([SIG_SR_PV[pair_SR_PV2plot[ii][jj]],NOS_SR_PV[pair_SR_PV2plot[ii][jj]]],bins=np.arange(-1,1,0.1),\
                        alpha=0.7,histtype='bar', stacked=True, \
                        color=['gray',[0.95,0.95,0.95]], edgecolor='gray', linewidth=2) 
        if round_p_PV[ii][jj]<0.05:
            ax[ii,jj].text(-0.95,3.5,'ttest p: '+str(round_p_PV[ii][jj]),\
                           ha='left', wrap=True,fontsize=18,weight='bold',color='r')
        else:
            ax[ii,jj].text(-0.95,3.5,'ttest p: '+str(round_p_PV[ii][jj]),\
                           ha='left', wrap=True,fontsize=18)
        ax[ii,jj].text(-0.95,5.5,'mean r: '+str(myf.roundP(np.nanmean(corr_SR_PV[pair_SR_PV2plot[ii][jj]]))), ha='left', wrap=True,fontsize=18)
        ax[ii,jj].set_title(pair_SR_PV2plot[ii][jj][0])
        ax[ii,jj].set_xlim(-1.1,1.1)
        ax[ii,jj].set_ylim(0,11)
        ax[ii,jj].axvline(0,color='k')
        
        
        if ii==5:
            ax[ii,jj].set_xlabel('Pearson corr coeff')
        else:
            ax[ii,jj].axes.get_xaxis().set_visible(False)
        
        if jj==0:
            ax[ii,0].set_ylabel(pair_SR_PV2plot[ii][jj][1])
        else:
            ax[ii,jj].axes.get_yaxis().set_visible(False)
 
plt.tight_layout()
plt.savefig('corr_psychometric_fit_results_SR.png')
plt.show()

In [ ]:
Mmedian_toffset,Mmax_toffset,Mmax_under100_toffset = [np.zeros((len(userids),len(sessionids))) for _ in range(3)]
ind1 = -1
for part in userids:
    ind1 += 1
    ind2 = -1
    for ses in sessionids:
        ind2 += 1
        if adf[(adf['userID_x']==part) & (adf['sessionID_x']==ses)].median_toffset.isnull().values.any():
            Mmedian_toffset[ind1,ind2] = np.nan
            Mmax_toffset[ind1,ind2] = np.nan
        else:
            Mmedian_toffset[ind1,ind2] = float(adf[(adf['userID_x']==part) & (adf['sessionID_x']==ses)].median_toffset)-300
            Mmax_toffset[ind1,ind2] = float(adf[(adf['userID_x']==part) & (adf['sessionID_x']==ses)].max_toffset)-300
        if Mmax_toffset[ind1,ind2] < 100:
            Mmax_under100_toffset[ind1,ind2] = Mmax_toffset[ind1,ind2]
        else:
            Mmax_under100_toffset[ind1,ind2] = np.nan

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig,ax = plt.subplots(ncols=3,figsize=(20,18))

img1 = ax[0].imshow(Mmedian_toffset,cmap="inferno")
divider = make_axes_locatable(ax[0])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(img1, cax=cax, orientation='vertical')
ax[0].set_yticks(np.arange(len(userids)))
ax[0].set_yticklabels(userids)
ax[0].set_ylabel('subject ID')
ax[0].set_xlabel('session number')
ax[0].set_title('median time offset - 300 ms')

img2 = ax[1].imshow(Mmax_toffset,cmap="inferno")
divider = make_axes_locatable(ax[1])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(img2, cax=cax, orientation='vertical')
ax[1].set_yticks(np.arange(len(userids)))
ax[1].set_yticklabels(userids)
ax[1].set_xlabel('session number')
ax[1].set_title('max time offset - 300 ms')

img3 = ax[2].imshow(Mmax_under100_toffset,cmap="inferno")
divider = make_axes_locatable(ax[2])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(img3, cax=cax, orientation='vertical')
ax[2].set_yticks(np.arange(len(userids)))
ax[2].set_yticklabels(userids)
ax[2].set_xlabel('session number')
ax[2].set_title('max time offset - 300 ms (up to 100)')

plt.tight_layout()
plt.savefig('/home/cbclab/Dropbox/postdocUPF/jatos_experiments/paper/t_offsetOT.png')
plt.show()

In [ ]:
df1011 = adf[adf['userID_x']==1011]
perf1011 = np.array(df1011['subj_perf_no'])
RTnoOK1011 = np.multiply(np.array(df1011['OKubj_RT_no']),np.array(df1011['medianRT']))
median_toffset1011 = np.array(df1011['median_toffset'])
max_toffset1011 = np.array(df1011['max_toffset'])

print(perf1011)

max_toffset1011 = max_toffset1011[~np.isnan(max_toffset1011)]
perf1011 = perf1011[~np.isnan(perf1011)]
slope, intercept, r_value, p_value, std_err  = stats.linregress(max_toffset1011,perf1011)

plt.plot(np.arange(300,370,5),intercept+slope*np.arange(300,370,5))
plt.fill_between(np.arange(300,370,5),intercept+slope*np.arange(300,370,5)-std_err, \
                   intercept+slope*np.arange(300,370,5)+std_err, color='b', alpha=0.2)

plt.scatter(max_toffset1011,perf1011)
plt.ylabel('acc (part 1011)')
plt.xlabel('max. time offset (ms)')
plt.text(300,93,'p-value: '+str(roundP(p_value)), ha='left', wrap=True,fontsize=18)
plt.tight_layout()
plt.savefig('/home/cbclab/Dropbox/postdocUPF/jatos_experiments/paper/acc_vs_toffset_part1011OT.png')
plt.show()

In [ ]:
RTnoOK1011 = np.multiply(np.array(df1011['OKubj_RT_no']),np.array(df1011['medianRT']))
max_toffset1011 = np.array(df1011['max_toffset'])

mask = ~np.isnan(max_toffset1011) & ~np.isnan(RTnoOK1011)

slope, intercept, r_value, p_value, std_err  = stats.linregress(max_toffset1011[mask],RTnoOK1011[mask])

plt.plot(np.arange(300,370,5),intercept+slope*np.arange(300,370,5))
plt.fill_between(np.arange(300,370,5),intercept+slope*np.arange(300,370,5)-std_err, \
                   intercept+slope*np.arange(300,370,5)+std_err, color='b', alpha=0.2)

plt.scatter(max_toffset1011,RTnoOK1011)
plt.ylabel('no RT (part 1011)')
plt.xlabel('max. time offset (ms)')
plt.text(300,930,'p-value: '+str(roundP(p_value)), ha='left', wrap=True,fontsize=18)
plt.tight_layout()
plt.savefig('/home/cbclab/Dropbox/postdocUPF/jatos_experiments/paper/RTnoOK_vs_toffset_part1011OT.png')
plt.show()

## Self-reports across participants

In [ ]:
key_name = ['mood','food', 'real_stress','sleep','Dsubj_optout_oo','Ssubj_optout_oo','Dsubj_perf_oo',\
           'Ssubj_perf_oo','subj_perf_no','OKubj_RT_no','Dsubj_RT_oo','Ssubj_RT_oo']

key_name2plot = [key_name[:int(len(key_name)/2)],key_name[int(len(key_name)/2):]]

ap, se = {},{}
for (var, op) in [(ap, np.nanmean), (se, myf.sem)]:
    for key in key_name:
        if key != 'sleep':
            var[key] = [op(np.array(adf[adf['sessionID_x']==sesid][key])) for sesid in sessionids]
        else:
            var[key] = [op(np.array(mdf[mdf['sessionID_x']==sesid][key])) for sesid in sessionids[0::2]]
session_labels = ['Th','Fr','Sa','Su','Mo','Tu','We','Th','Fr','Sa']

In [ ]:
fig, ax = plt.subplots(6,2,figsize=(18,18))
plt.subplots_adjust(hspace = 0.3)  

for i in range(6):
    for j in range(2):
        lower_limit = np.min(ap[key_name2plot[j][i]])-np.max(se[key_name2plot[j][i]])
        upper_limit = np.max(ap[key_name2plot[j][i]])+np.max(se[key_name2plot[j][i]])
        ax[i,j].fill_between([4,8],lower_limit,upper_limit,color=[0.9,0.9,0.9])
        ax[i,j].fill_between([18,20],lower_limit,upper_limit,color=[0.9,0.9,0.9])
        ax[i,j].set_xticks(np.arange(1,21)[::2])
        ax[i,j].set_xticklabels(session_labels,rotation=45)
        ax[i,j].set_ylabel(key_name2plot[j][i])
        if key_name2plot[j][i]!='sleep':
            ax[i,j].plot(np.arange(1,len(ap[key_name2plot[j][i]])+1), ap[key_name2plot[j][i]], color = 'b')
            ax[i,j].fill_between(np.arange(1,len(ap[key_name2plot[j][i]])+1),np.array(ap[key_name2plot[j][i]])-se[key_name2plot[j][i]], \
                       np.array(ap[key_name2plot[j][i]])+se[key_name2plot[j][i]], alpha=0.2)   
        else:
            ax[i,j].plot(np.arange(1,21)[::2],ap[key_name2plot[j][i]],color='b')
            ax[i,j].fill_between(np.arange(1,21)[::2],\
                       np.array(ap[key_name2plot[j][i]])-se[key_name2plot[j][i]], \
                       np.array(ap[key_name2plot[j][i]])+se[key_name2plot[j][i]], alpha=0.2)  
        
          

plt.show()

In [ ]:
# write the result in file
filename_ap=path_results+'mean_across_participants.json'
filename_se=path_results+'se_across_participants.json'
# Serializing json  
json_object_ap = json.dumps(ap) 
json_object_se = json.dumps(se)

# Writing to sample.json 
with open(filename_ap, "w") as outfile: 
    outfile.write(json_object_ap) 
with open(filename_se, "w") as outfile: 
    outfile.write(json_object_se) 

- Tu M: Drama session--> introduction of failure second link
- We A: Third link with limited last noise staircase within the range 84-92.

## Relationship between reports

Avoiding the participant who always response the same value...

In [ ]:
corr_mo_fo,corr_mo_st,corr_st_fo = [[] for _ in range(3)]
corr_sl_mo, corr_sl_fo, corr_sl_st = [[] for _ in range(3)]

ind = -1
for part in userids:
    ind += 1
    subset = adf[adf['userID_x']==part]
    mood = np.array(subset['mood'])
    stress = np.array(subset['real_stress'])
    food = np.array(subset['food'])
    # delete nan values
    mood = mood[~np.isnan(mood)]
    stress = stress[~np.isnan(stress)]
    food = food[~np.isnan(food)]
    # avoid part 1010 who always answered food=1
    if part!=1010:    
        corr_mo_fo.append(np.corrcoef(mood,food)[0][1])
        corr_mo_st.append(np.corrcoef(mood,stress)[0][1])
        corr_st_fo.append(np.corrcoef(stress,food)[0][1])
    
    # morning
    subset_sl = mdf[mdf['userID_x']==part]
    sleep = np.array(subset_sl['sleep'])
    mood_mor = np.array(subset_sl['mood'])
    food_mor = np.array(subset_sl['food'])
    stress_mor = np.array(subset_sl['real_stress'])
    # delete nan values
    sleep = sleep[~np.isnan(sleep)]
    mood_mor = mood_mor[~np.isnan(mood_mor)]
    stress_mor = stress_mor[~np.isnan(stress_mor)]
    food_mor = food_mor[~np.isnan(food_mor)]       
    # avoid part 1005 who always answered sleep=1
    if part!=1005:
        corr_sl_mo.append(np.corrcoef(sleep,mood_mor)[0][1])
        corr_sl_st.append(np.corrcoef(sleep,stress_mor)[0][1])
    # avoid part 1010 who always answered food=1
    if part!=1005 and part!=1010:
        corr_sl_fo.append(np.corrcoef(sleep,food_mor)[0][1])

p_mo_fo = stats.ttest_1samp(corr_mo_fo,0)[1]
p_mo_st = stats.ttest_1samp(corr_mo_st,0)[1]
p_st_fo = stats.ttest_1samp(corr_st_fo,0)[1]
p_sl_mo = stats.ttest_1samp(corr_sl_mo,0)[1]
p_sl_fo = stats.ttest_1samp(corr_sl_fo,0)[1]
p_sl_st = stats.ttest_1samp(corr_sl_st,0)[1]

ptable = [['','mood','food','stress','sleep'],
          ['mood',0,roundP(p_mo_fo),roundP(p_mo_st),roundP(p_sl_mo)],
         ['food',roundP(p_mo_fo),0,roundP(p_st_fo),roundP(p_sl_fo)],
         ['stress',roundP(p_mo_st),roundP(p_st_fo),0,roundP(p_sl_st)],
         ['sleep',roundP(p_sl_mo),roundP(p_sl_fo),roundP(p_sl_st),0]]

display(HTML(tabulate.tabulate(ptable, tablefmt='html')))

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,3))
ax[0].hist(corr_mo_st,color='r')
ax[0].set_title("mood & stress")
ax[0].set_xlabel('Pearson correlation coef',fontsize=18)
ax[0].set_ylabel('Counts',fontsize=18)
ax[0].text(0.05,3,'ttest p: '+str(roundP(p_mo_st)), ha='left', wrap=True,fontsize=18)
ax[0].text(0.05,2.5,'mean p: '+str(roundP(np.nanmean(corr_mo_st))), ha='left', wrap=True,fontsize=18)
ax[0].set_xlim(-1.1,1.1)
ax[0].axvline(0,color='k')

ax[1].hist(corr_mo_fo,color='y')
ax[1].set_title("mood & food")
ax[1].set_xlabel('Pearson correlation coef',fontsize=18)
ax[1].text(0.05,3,'ttest p: '+str(roundP(p_mo_fo)), ha='left', wrap=True,fontsize=18)
ax[1].text(0.05,2.5,'mean p: '+str(roundP(np.nanmean(corr_mo_fo))), ha='left', wrap=True,fontsize=18)
ax[1].set_xlim(-1.1,1.1)
ax[1].axvline(0,color='k')

ax[2].hist(corr_st_fo,color='g')
ax[2].set_title("stress & food")
ax[2].set_xlabel('Pearson correlation coef',fontsize=18)
ax[2].text(0.05,3,'ttest p: '+str(roundP(p_st_fo)), ha='left', wrap=True,fontsize=18)
ax[2].text(0.05,2.5,'mean p: '+str(roundP(np.nanmean(corr_st_fo))), ha='left', wrap=True,fontsize=18)
ax[2].set_xlim(-1.1,1.1)
ax[2].axvline(0,color='k')

plt.show()

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,3))
ax[0].hist(corr_sl_mo,color='r')
ax[0].set_title("mood & sleep")
ax[0].set_xlabel('Pearson correlation coef',fontsize=18)
ax[0].set_ylabel('Counts',fontsize=18)
ax[0].text(0.05,3,'ttest p: '+str(roundP(p_sl_mo)), ha='left', wrap=True,fontsize=18)
ax[0].text(0.05,2.5,'mean p: '+str(roundP(np.nanmean(corr_sl_mo))), ha='left', wrap=True,fontsize=18)
ax[0].set_xlim(-1.1,1.1)
ax[0].axvline(0,color='k')

ax[1].hist(corr_sl_fo,color='y')
ax[1].set_title("sleep & food")
ax[1].set_xlabel('Pearson correlation coef',fontsize=18)
ax[1].text(0.05,3,'ttest p: '+str(roundP(p_sl_fo)), ha='left', wrap=True,fontsize=18)
ax[1].text(0.05,2.5,'mean p: '+str(roundP(np.nanmean(corr_sl_fo))), ha='left', wrap=True,fontsize=18)
ax[1].set_xlim(-1.1,1.1)
ax[1].axvline(0,color='k')

ax[2].hist(corr_sl_st,color='g')
ax[2].set_title("stress & sleep")
ax[2].set_xlabel('Pearson correlation coef',fontsize=18)
ax[2].text(0.05,3,'ttest p: '+str(roundP(p_sl_st)), ha='left', wrap=True,fontsize=18)
ax[2].text(0.05,2.5,'mean p: '+str(roundP(np.nanmean(corr_sl_st))), ha='left', wrap=True,fontsize=18)
ax[2].set_xlim(-1.1,1.1)
ax[2].axvline(0,color='k')

plt.show()

In [ ]:
matrixR = [[1,np.nanmean(corr_mo_st),np.nanmean(corr_sl_mo),np.nanmean(corr_mo_fo)],\
           [np.nanmean(corr_mo_st),1,np.nanmean(corr_sl_st),np.nanmean(corr_st_fo)],\
           [np.nanmean(corr_sl_mo),np.nanmean(corr_sl_st),1,np.nanmean(corr_sl_fo)],\
           [np.nanmean(corr_mo_fo),np.nanmean(corr_st_fo),np.nanmean(corr_sl_fo),1]]

In [ ]:
print(matrixR)

In [ ]:
pmatrix = [[0,p_mo_st,p_sl_mo,p_mo_fo],
         [p_mo_st,0,p_sl_st,p_st_fo],
         [p_sl_mo,p_sl_st,0,p_sl_fo],
         [p_mo_fo,p_st_fo,p_sl_fo,0]]
print(pmatrix)

In [ ]:
plt.imshow(matrixR,cmap='seismic',interpolation='nearest', vmin=-1, vmax=1)
plt.xticks(np.arange(4),['mood','stress','sleep','food'],rotation = 45)
plt.yticks(np.arange(4),['mood','stress','sleep','food'])
plt.colorbar()
plt.tight_layout()
plt.title('Orientation task')
plt.savefig('/home/cbclab/Dropbox/postdocUPF/jatos_experiments/paper/reports_corr_gabor.png')
plt.show()

In [ ]:
r_subjNOS_fo_mo,r_subjNOS_st_mo,r_subjNOS_sl_mo = [[] for _ in range(3)]
r_subjSIG_fo_mo,r_subjSIG_st_mo,r_subjSIG_sl_mo = [[] for _ in range(3)]
ind = -1
for part in userids:
    ind += 1
    subset = adf[adf['userID_x']==part]
    mood = np.array(subset['mood'])
    stress = np.array(subset['real_stress'])
    food = np.array(subset['food'])
        
    # food & optout
    slope_fo, intercept_fo, r_value_fo, p_value_fo, std_err_fo = stats.linregress(food,mood)
    if p_value_fo<0.05:
        r_subjSIG_fo_mo.append(r_value_fo)
    else:
        r_subjNOS_fo_mo.append(r_value_fo)
        
    # stress & optout
    slope_st, intercept_st, r_value_st, p_value_st, std_err_st = stats.linregress(stress,mood)
    if p_value_st<0.05:
        r_subjSIG_st_mo.append(r_value_st)
    else:
        r_subjNOS_st_mo.append(r_value_st)
        
    # morning
    subset_mor = mdf[mdf['userID_x']==part]
    sleep = np.array(subset_mor['sleep'])
    mood_mor = np.array(subset_mor['mood'])
    
    # sleep & mood
    slope_sl, intercept_sl, r_value_sl, p_value_sl, std_err_sl = stats.linregress(sleep,mood_mor)
    if p_value_sl<0.05:
        r_subjSIG_sl_mo.append(r_value_sl)
    else:
        r_subjNOS_sl_mo.append(r_value_sl)

In [ ]:
# write the result in file
dict_ = {
    "r_subjSIG_st_mo":list(r_subjSIG_st_mo),
    "r_subjNOS_st_mo":list(r_subjNOS_st_mo),
    "r_subjSIG_fo_mo":list(r_subjSIG_fo_mo),
    "r_subjNOS_fo_mo":list(r_subjNOS_fo_mo),
    "r_subjSIG_sl_mo":list(r_subjSIG_sl_mo),
    "r_subjNOS_sl_mo":list(r_subjNOS_sl_mo),
}
# Serializing json  
json_object = json.dumps(dict_) 

# Writing to sample.json 
with open('/home/cbclab/Dropbox/postdocUPF/jatos_experiments/gabor/notebooks/results_tanda_1/corr_reports_reports.json', "w") as outfile: 
    outfile.write(json_object) 

## Correlation between staircase and reports

In [ ]:
corr_mo_stsigma,corr_st_stsigma,corr_fo_stsigma,corr_sl_stsigma = [[] for _ in range(4)]
corr_mo_stbias,corr_st_stbias,corr_fo_stbias,corr_sl_stbias = [[] for _ in range(4)]
ind = -1
for part in userids:
    ind += 1
    subset = adf[adf['userID_x']==part]
    mood = np.array(subset['mood'])
    stress = np.array(subset['real_stress'])
    food = np.array(subset['food'])
    st_sigma = np.array(subset['stair_sigma'])
    st_bias = np.array(subset['stair_bias'])
    if not m.isnan(np.corrcoef(mood,st_sigma)[0][1]):
        corr_mo_stsigma.append(np.corrcoef(mood,st_sigma)[0][1])
    if not m.isnan(np.corrcoef(stress,st_sigma)[0][1]):
        corr_st_stsigma.append(np.corrcoef(stress,st_sigma)[0][1])
    if not m.isnan(np.corrcoef(food,st_sigma)[0][1]):
        corr_fo_stsigma.append(np.corrcoef(food,st_sigma)[0][1])
        
    if not m.isnan(np.corrcoef(mood,st_bias)[0][1]):
        corr_mo_stbias.append(np.corrcoef(mood,st_bias)[0][1])
    if not m.isnan(np.corrcoef(stress,st_bias)[0][1]):
        corr_st_stbias.append(np.corrcoef(stress,st_bias)[0][1])
    if not m.isnan(np.corrcoef(food,st_bias)[0][1]):
        corr_fo_stbias.append(np.corrcoef(food,st_bias)[0][1])
        
    subset_sl = mdf[mdf['userID_x']==part]
    sleep = np.array(subset_sl['sleep'])
    st_sigma_mor = np.array(subset_sl['stair_sigma'])
    if not m.isnan(np.corrcoef(sleep,st_sigma_mor)[0][1]):
        corr_sl_stsigma.append(np.corrcoef(sleep,st_sigma_mor)[0][1])
    st_bias_mor = np.array(subset_sl['stair_bias'])
    if not m.isnan(np.corrcoef(sleep,st_bias_mor)[0][1]):
        corr_sl_stbias.append(np.corrcoef(sleep,st_bias_mor)[0][1])

p_mo_stsigma = stats.ttest_1samp(corr_mo_stsigma,0)[1]
p_st_stsigma = stats.ttest_1samp(corr_st_stsigma,0)[1]
p_fo_stsigma = stats.ttest_1samp(corr_fo_stsigma,0)[1]
p_sl_stsigma = stats.ttest_1samp(corr_sl_stsigma,0)[1]

p_mo_stbias = stats.ttest_1samp(corr_mo_stbias,0)[1]
p_st_stbias = stats.ttest_1samp(corr_st_stbias,0)[1]
p_fo_stbias = stats.ttest_1samp(corr_fo_stbias,0)[1]
p_sl_stbias = stats.ttest_1samp(corr_sl_stbias,0)[1]

ptable = [['staircase','noise','bias'],
          ['mood',roundP(p_mo_stsigma),roundP(p_mo_stbias)],
         ['food',roundP(p_fo_stsigma),roundP(p_st_stbias)],
         ['stress',roundP(p_st_stsigma),roundP(p_fo_stbias)],
         ['sleep',roundP(p_sl_stsigma),roundP(p_sl_stbias)]]

display(HTML(tabulate.tabulate(ptable, tablefmt='html')))

In [ ]:
corr_mo_stbias,corr_st_stbias,corr_fo_stbias,corr_sl_stbias = [[] for _ in range(4)]
ind = -1
for part in userids:
    ind += 1
    subset = adf[adf['userID_x']==part]
    mood = np.array(subset['mood'])
    stress = np.array(subset['real_stress'])
    food = np.array(subset['food'])
    st_bias = np.array(subset['stair_bias'])
    if not m.isnan(np.corrcoef(mood,st_bias)[0][1]):
        corr_mo_stbias.append(np.corrcoef(mood,st_bias)[0][1])
    if not m.isnan(np.corrcoef(stress,st_bias)[0][1]):
        corr_st_stbias.append(np.corrcoef(stress,st_bias)[0][1])
    if not m.isnan(np.corrcoef(food,st_bias)[0][1]):
        corr_fo_stbias.append(np.corrcoef(food,st_bias)[0][1])
    subset_sl = mdf[mdf['userID_x']==part]
    sleep = np.array(subset_sl['sleep'])
    st_bias_mor = np.array(subset_sl['stair_bias'])
    if not m.isnan(np.corrcoef(sleep,st_bias_mor)[0][1]):
        corr_sl_stbias.append(np.corrcoef(sleep,st_bias_mor)[0][1])

p_mo_stbias = stats.ttest_1samp(corr_mo_stbias,0)[1]
p_st_stbias = stats.ttest_1samp(corr_st_stbias,0)[1]
p_fo_stbias = stats.ttest_1samp(corr_fo_stbias,0)[1]
p_sl_stbias = stats.ttest_1samp(corr_sl_stbias,0)[1]

## Correlation between performance and reports

In [ ]:
Dcorr_acc_mo,Dcorr_acc_fo,Dcorr_acc_sl,Dcorr_acc_st = [[] for _ in range(4)]
Scorr_acc_mo,Scorr_acc_fo,Scorr_acc_sl,Scorr_acc_st = [[] for _ in range(4)]
NOcorr_acc_mo,NOcorr_acc_fo,NOcorr_acc_sl,NOcorr_acc_st = [[] for _ in range(4)]
ind = -1
for part in userids:
    ind += 1
    subset = adf[adf['userID_x']==part]
    mood = np.array(subset['mood'])
    stress = np.array(subset['real_stress'])
    food = np.array(subset['food'])
    Dperf = np.array(subset['Dsubj_perf_oo'])
    Sperf = np.array(subset['Ssubj_perf_oo'])
    NOperf = np.array(subset['subj_perf_no'])
    if not m.isnan(np.corrcoef(mood,Dperf)[0][1]):
        Dcorr_acc_mo.append(np.corrcoef(mood,Dperf)[0][1])
    if not m.isnan(np.corrcoef(mood,Sperf)[0][1]):
        Scorr_acc_mo.append(np.corrcoef(mood,Sperf)[0][1])
    if not m.isnan(np.corrcoef(mood,NOperf)[0][1]):
        NOcorr_acc_mo.append(np.corrcoef(mood,NOperf)[0][1])
    if not m.isnan(np.corrcoef(stress,Dperf)[0][1]):
        Dcorr_acc_st.append(np.corrcoef(stress,Dperf)[0][1])
    if not m.isnan(np.corrcoef(stress,Sperf)[0][1]):
        Scorr_acc_st.append(np.corrcoef(stress,Sperf)[0][1])
    if not m.isnan(np.corrcoef(stress,NOperf)[0][1]):
        NOcorr_acc_st.append(np.corrcoef(stress,NOperf)[0][1])
    if not m.isnan(np.corrcoef(food,Dperf)[0][1]):
        Dcorr_acc_fo.append(np.corrcoef(food,Dperf)[0][1])
    if not m.isnan(np.corrcoef(food,Sperf)[0][1]):
        Scorr_acc_fo.append(np.corrcoef(food,Sperf)[0][1])
    if not m.isnan(np.corrcoef(food,NOperf)[0][1]):
        NOcorr_acc_fo.append(np.corrcoef(food,NOperf)[0][1])
    # morning sessions
    subset_mor = mdf[mdf['userID_x']==part]
    sleep = np.array(subset_mor['sleep'])
    if not m.isnan(np.corrcoef(sleep,subset_mor['Dsubj_perf_oo'])[0][1]):
        Dcorr_acc_sl.append(np.corrcoef(sleep,subset_mor['Dsubj_perf_oo'])[0][1])
    if not m.isnan(np.corrcoef(sleep,subset_mor['Ssubj_perf_oo'])[0][1]):
        Scorr_acc_sl.append(np.corrcoef(sleep,subset_mor['Ssubj_perf_oo'])[0][1])
    if not m.isnan(np.corrcoef(sleep,subset_mor['subj_perf_no'])[0][1]):
        NOcorr_acc_sl.append(np.corrcoef(sleep,subset_mor['subj_perf_no'])[0][1])
        
Dp_acc_mo = stats.ttest_1samp(Dcorr_acc_mo,0)[1]    
Dp_acc_fo = stats.ttest_1samp(Dcorr_acc_fo,0)[1]  
Dp_acc_sl = stats.ttest_1samp(Dcorr_acc_sl,0)[1]  
Dp_acc_st = stats.ttest_1samp(Dcorr_acc_st,0)[1]   

Sp_acc_mo = stats.ttest_1samp(Scorr_acc_mo,0)[1]    
Sp_acc_fo = stats.ttest_1samp(Scorr_acc_fo,0)[1]  
Sp_acc_sl = stats.ttest_1samp(Scorr_acc_sl,0)[1]  
Sp_acc_st = stats.ttest_1samp(Scorr_acc_st,0)[1]  

NOp_acc_mo = stats.ttest_1samp(NOcorr_acc_mo,0)[1]    
NOp_acc_fo = stats.ttest_1samp(NOcorr_acc_fo,0)[1]  
NOp_acc_sl = stats.ttest_1samp(NOcorr_acc_sl,0)[1]  
NOp_acc_st = stats.ttest_1samp(NOcorr_acc_st,0)[1] 

ptable = [['',' perf in DO',' perf in SO', 'perf in NO'],
          ['mood',roundP(Dp_acc_mo),roundP(Sp_acc_mo),roundP(NOp_acc_mo)],
         ['food',roundP(Dp_acc_fo),roundP(Sp_acc_fo),roundP(NOp_acc_fo)],
         ['stress',roundP(Dp_acc_st),roundP(Sp_acc_st),roundP(NOp_acc_st)],
         ['sleep',roundP(Dp_acc_sl),roundP(Sp_acc_sl),roundP(NOp_acc_sl)]]

#display(Latex(r"\newpage"))

display(HTML(tabulate.tabulate(ptable, tablefmt='html')))
#print(tabulate.tabulate(ptable))
#pt.PrettyTable(ptable)


In [ ]:
print([np.nanmean(NOcorr_acc_mo),np.nanmean(NOcorr_acc_st),np.nanmean(NOcorr_acc_sl),np.nanmean(NOcorr_acc_fo)])

In [ ]:
print([np.nanmean(NOp_acc_mo),np.nanmean(NOp_acc_st),np.nanmean(NOp_acc_sl),np.nanmean(NOp_acc_fo)])

In [ ]:
fig, ax = plt.subplots(figsize=(5,3))
ax.hist(Scorr_acc_sl,color='b')
ax.text(0.5,0.5,'ttest p: '+str(roundP(Sp_acc_sl)), ha='left', wrap=True,fontsize=18)
ax.text(0.5,1.5,'mean r: '+str(roundP(np.nanmean(Scorr_acc_sl))), ha='left', wrap=True,fontsize=18)
ax.set_title("SO perf & sleep")
ax.set_xlabel('Pearson corr coeff')
ax.set_ylabel('Counts')
ax.set_xlim(-1.1,1.1)
ax.axvline(0,color='k')
plt.show()

## Correlation between optout and reports

In [ ]:
Dcorr_oo_mo,Dcorr_oo_fo,Dcorr_oo_sl,Dcorr_oo_st = [[] for _ in range(4)]
Scorr_oo_mo,Scorr_oo_fo,Scorr_oo_sl,Scorr_oo_st = [[] for _ in range(4)]
ind = -1
for part in userids:
    ind += 1
    subset = adf[adf['userID_x']==part]
    mood = np.array(subset['mood'])
    stress = np.array(subset['real_stress'])
    food = np.array(subset['food'])
    DO = np.array(subset['Dsubj_optout_oo'])
    SO = np.array(subset['Ssubj_optout_oo'])
    if not m.isnan(np.corrcoef(mood,DO)[0][1]):
        Dcorr_oo_mo.append(np.corrcoef(mood,DO)[0][1])
    if not m.isnan(np.corrcoef(mood,SO)[0][1]):
        Scorr_oo_mo.append(np.corrcoef(mood,SO)[0][1])
    if not m.isnan(np.corrcoef(stress,DO)[0][1]):
        Dcorr_oo_st.append(np.corrcoef(stress,DO)[0][1])
    if not m.isnan(np.corrcoef(stress,SO)[0][1]):
        Scorr_oo_st.append(np.corrcoef(stress,SO)[0][1])
    if not m.isnan(np.corrcoef(food,DO)[0][1]):
        Dcorr_oo_fo.append(np.corrcoef(food,DO)[0][1])
    if not m.isnan(np.corrcoef(food,SO)[0][1]):
        Scorr_oo_fo.append(np.corrcoef(food,SO)[0][1])
    # morning
    subset_mor = mdf[mdf['userID_x']==part]
    sleep = np.array(subset_mor['sleep'])
    DOmor = subset_mor['Dsubj_optout_oo']
    SOmor = subset_mor['Ssubj_optout_oo']
    if not m.isnan(np.corrcoef(sleep,DOmor)[0][1]):
        Dcorr_oo_sl.append(np.corrcoef(sleep,DOmor)[0][1])
    if not m.isnan(np.corrcoef(sleep,SOmor)[0][1]):
        Scorr_oo_sl.append(np.corrcoef(sleep,SOmor)[0][1])

Dp_oo_mo = stats.ttest_1samp(Dcorr_oo_mo,0)[1]    
Dp_oo_fo = stats.ttest_1samp(Dcorr_oo_fo,0)[1]  
Dp_oo_sl = stats.ttest_1samp(Dcorr_oo_sl,0)[1]  
Dp_oo_st = stats.ttest_1samp(Dcorr_oo_st,0)[1]   

Sp_oo_mo = stats.ttest_1samp(Scorr_oo_mo,0)[1]    
Sp_oo_fo = stats.ttest_1samp(Scorr_oo_fo,0)[1]  
Sp_oo_sl = stats.ttest_1samp(Scorr_oo_sl,0)[1]  
Sp_oo_st = stats.ttest_1samp(Scorr_oo_st,0)[1]  

ptable = [['','DO','SO'],
          ['mood',roundP(Dp_oo_mo),roundP(Sp_oo_mo)],
         ['food',roundP(Dp_oo_fo),roundP(Sp_oo_fo)],
         ['stress',roundP(Dp_oo_st),roundP(Sp_oo_st)],
         ['sleep',roundP(Dp_oo_sl),roundP(Sp_oo_sl)]]

#display(Latex(r"\newpage"))

display(HTML(tabulate.tabulate(ptable, tablefmt='html')))
#print(tabulate.tabulate(ptable))
#pt.PrettyTable(ptable)

In [ ]:
print([np.nanmean(Dcorr_oo_mo),np.nanmean(Dcorr_oo_st),np.nanmean(Dcorr_oo_sl),np.nanmean(Dcorr_oo_fo)])

In [ ]:
print([np.nanmean(Dp_oo_mo),np.nanmean(Dp_oo_st),np.nanmean(Dp_oo_sl),np.nanmean(Dp_oo_fo)])

In [ ]:
print([np.nanmean(Scorr_oo_mo),np.nanmean(Scorr_oo_st),np.nanmean(Scorr_oo_sl),np.nanmean(Scorr_oo_fo)])

In [ ]:
print([np.nanmean(Sp_oo_mo),np.nanmean(Sp_oo_st),np.nanmean(Sp_oo_sl),np.nanmean(Sp_oo_fo)])

In [ ]:
fig, ax = plt.subplots(4,2,figsize=(10,14))
plt.subplots_adjust(wspace = 0.3)
plt.subplots_adjust(hspace = 0.5)
ax[0,0].hist(Dcorr_oo_mo,color='r')
ax[0,0].text(0.2,2.5,'ttest p: '+str(roundP(Dp_oo_mo)), ha='left', wrap=True,fontsize=18)
ax[0,0].text(0.2,1.5,'mean r: '+str(roundP(np.nanmean(Dcorr_oo_mo))), ha='left', wrap=True,fontsize=18)
ax[0,0].set_title("DO optout & mood")
ax[0,0].set_ylabel('Counts')
ax[0,0].set_xlim(-1.1,1.1)
ax[0,0].axvline(0,color='k')

ax[0,1].hist(Scorr_oo_mo,color='b')
ax[0,1].text(0.2,2.5,'ttest p: '+str(roundP(Sp_oo_mo)), ha='left', wrap=True,fontsize=18)
ax[0,1].text(0.2,1.5,'mean r: '+str(roundP(np.nanmean(Scorr_oo_mo))), ha='left', wrap=True,fontsize=18)
ax[0,1].set_title("SO optout & mood")
ax[0,1].set_xlim(-1.1,1.1)
ax[0,1].axvline(0,color='k')

ax[1,0].hist(Dcorr_oo_fo,color='r')
ax[1,0].text(0.2,2.5,'ttest p: '+str(roundP(Dp_oo_fo)), ha='left', wrap=True,fontsize=18)
ax[1,0].text(0.2,1.5,'mean r: '+str(roundP(np.nanmean(Dcorr_oo_fo))), ha='left', wrap=True,fontsize=18)
ax[1,0].set_title("DO optout & food")
ax[1,0].set_ylabel('Counts')
ax[1,0].set_xlim(-1.1,1.1)
ax[1,0].axvline(0,color='k')

ax[1,1].hist(Scorr_oo_fo,color='b')
ax[1,1].text(0.2,2.5,'ttest p: '+str(roundP(Sp_oo_fo)), ha='left', wrap=True,fontsize=18)
ax[1,1].text(0.2,1.5,'mean r: '+str(roundP(np.nanmean(Scorr_oo_fo))), ha='left', wrap=True,fontsize=18)
ax[1,1].set_title("SO optout & food")
ax[1,1].set_xlim(-1.1,1.1)
ax[1,1].axvline(0,color='k')

ax[2,0].hist(Dcorr_oo_st,color='r')
ax[2,0].text(0.2,2.5,'ttest p: '+str(roundP(Dp_oo_st)), ha='left', wrap=True,fontsize=18)
ax[2,0].text(0.2,1.5,'mean r: '+str(roundP(np.nanmean(Dcorr_oo_st))), ha='left', wrap=True,fontsize=18)
ax[2,0].set_title("DO optout & stress")
ax[2,0].set_ylabel('Counts')
ax[2,0].set_xlim(-1.1,1.1)
ax[2,0].axvline(0,color='k')

ax[2,1].hist(Scorr_oo_st,color='b')
ax[2,1].text(0.2,2.5,'ttest p: '+str(roundP(Sp_oo_st)), ha='left', wrap=True,fontsize=18)
ax[2,1].text(0.2,1.5,'mean r: '+str(roundP(np.nanmean(Scorr_oo_st))), ha='left', wrap=True,fontsize=18)
ax[2,1].set_title("SO optout & stress")
ax[2,1].set_xlim(-1.1,1.1)
ax[2,1].axvline(0,color='k')

ax[3,0].hist(Dcorr_oo_sl,color='r')
ax[3,0].text(0.2,2.5,'ttest p: '+str(roundP(Dp_oo_sl)), ha='left', wrap=True,fontsize=18)
ax[3,0].text(0.2,1.5,'mean r: '+str(roundP(np.nanmean(Dcorr_oo_sl))), ha='left', wrap=True,fontsize=18)
ax[3,0].set_title("DO optout & sleep")
ax[3,0].set_xlabel('Pearson corr coeff')
ax[3,0].set_ylabel('Counts')
ax[3,0].set_xlim(-1.1,1.1)
ax[3,0].axvline(0,color='k')

ax[3,1].hist(Scorr_oo_sl,color='b')
ax[3,1].text(0.2,2.5,'ttest p: '+str(roundP(Sp_oo_sl)), ha='left', wrap=True,fontsize=18)
ax[3,1].text(0.2,1.5,'mean r: '+str(roundP(np.nanmean(Scorr_oo_sl))), ha='left', wrap=True,fontsize=18)
ax[3,1].set_title("SO optout & sleep")
ax[3,1].set_xlabel('Pearson corr coeff')
ax[3,1].set_xlim(-1.1,1.1)
ax[3,1].axvline(0,color='k')

plt.show()

In [ ]:
p_subj_mo_do,r_subj_mo_do,p_subj_fo_do,r_subj_fo_do,p_subj_st_do,r_subj_st_do,p_subj_sl_do,r_subj_sl_do = [[] for _ in range(8)]
p_subj_mo_so,r_subj_mo_so,p_subj_fo_so,r_subj_fo_so,p_subj_st_so,r_subj_st_so,p_subj_sl_so,r_subj_sl_so = [[] for _ in range(8)]
ind = -1
for part in userids:
    ind += 1
    subset = adf[adf['userID_x']==part]
    mood = np.array(subset['mood'])
    stress = np.array(subset['real_stress'])
    food = np.array(subset['food'])
    DO = np.array(subset['Dsubj_optout_oo'])
    SO = np.array(subset['Ssubj_optout_oo'])
    slope_mo, intercept_mo, r_value_mo, p_value_mo, std_err_mo = stats.linregress(mood,DO)
    if p_value_mo<0.09:
        print('correlation btw mood and DO for participant ', part, ' with p_value ',p_value_mo,' and r_value ',r_value_mo)
    p_subj_mo_do.append(p_value_mo)
    r_subj_mo_do.append(r_value_mo)
    Sslope_mo, Sintercept_mo, Sr_value_mo, Sp_value_mo, Sstd_err_mo = stats.linregress(mood,SO)
    p_subj_mo_so.append(Sp_value_mo)
    r_subj_mo_so.append(Sr_value_mo)
    slope_fo, intercept_fo, r_value_fo, p_value_fo, std_err_fo = stats.linregress(food,DO)
    p_subj_fo_do.append(p_value_fo)
    r_subj_fo_do.append(r_value_fo)
    Sslope_fo, Sintercept_fo, Sr_value_fo, Sp_value_fo, Sstd_err_fo = stats.linregress(food,SO)
    p_subj_fo_so.append(Sp_value_fo)
    r_subj_fo_so.append(Sr_value_fo)
    slope_st, intercept_st, r_value_st, p_value_st, std_err_st = stats.linregress(stress,DO)
    p_subj_st_do.append(p_value_st)
    r_subj_st_do.append(r_subj_st_do)
    Sslope_st, Sintercept_st, Sr_value_st, Sp_value_st, Sstd_err_st = stats.linregress(stress,SO)
    p_subj_st_so.append(Sp_value_st)
    r_subj_st_so.append(Sr_value_st)
    # morning
    subset_mor = mdf[mdf['userID_x']==part]
    sleep = np.array(subset_mor['sleep'])
    DOmor = subset_mor['Dsubj_optout_oo']
    SOmor = subset_mor['Ssubj_optout_oo']
    slope_sl, intercept_sl, r_value_sl, p_value_sl, std_err_sl = stats.linregress(sleep,DOmor)
    p_subj_sl_do.append(p_value_sl)
    r_subj_sl_do.append(r_value_sl)
    Sslope_sl, Sintercept_sl, Sr_value_sl, Sp_value_sl, Sstd_err_sl = stats.linregress(sleep,SOmor)
    p_subj_sl_so.append(Sp_value_sl)
    r_subj_sl_so.append(Sstd_err_sl)

ptable_do = [p_subj_mo_do,p_subj_fo_do,p_subj_st_do,p_subj_sl_do]
ptable_so = [p_subj_mo_so,p_subj_fo_so,p_subj_st_so,p_subj_sl_so]

#display(HTML(tabulate.tabulate(ptable_do, tablefmt='html')))   
#display(HTML(tabulate.tabulate(ptable_so, tablefmt='html')))  

## Correlation between response time and reports

In [ ]:
Dcorr_RT_mo,Dcorr_RT_fo,Dcorr_RT_sl,Dcorr_RT_st = [[] for _ in range(4)]
Scorr_RT_mo,Scorr_RT_fo,Scorr_RT_sl,Scorr_RT_st = [[] for _ in range(4)]
NOcorr_RT_mo,NOcorr_RT_fo,NOcorr_RT_sl,NOcorr_RT_st = [[] for _ in range(4)]
ind = -1
for part in userids:
    ind += 1
    subset = adf[adf['userID_x']==part]
    mood = np.array(subset['mood'])
    stress = np.array(subset['real_stress'])
    food = np.array(subset['food'])
    DRT = np.array(subset['OKDsubj_RT_oo'])
    SRT = np.array(subset['OKSsubj_RT_oo'])
    NORT = np.array(subset['OKubj_RT_no'])
    if not m.isnan(np.corrcoef(mood,DRT)[0][1]):
        Dcorr_RT_mo.append(np.corrcoef(mood,DRT)[0][1])
    if not m.isnan(np.corrcoef(mood,SRT)[0][1]):
        Scorr_RT_mo.append(np.corrcoef(mood,SRT)[0][1])
    if not m.isnan(np.corrcoef(mood,NORT)[0][1]):
        NOcorr_RT_mo.append(np.corrcoef(mood,NORT)[0][1])
    if not m.isnan(np.corrcoef(stress,DRT)[0][1]):
        Dcorr_RT_st.append(np.corrcoef(stress,DRT)[0][1])
    if not m.isnan(np.corrcoef(stress,SRT)[0][1]):
        Scorr_RT_st.append(np.corrcoef(stress,SRT)[0][1])
    if not m.isnan(np.corrcoef(stress,NORT)[0][1]):
        NOcorr_RT_st.append(np.corrcoef(stress,NORT)[0][1])
    if not m.isnan(np.corrcoef(food,DRT)[0][1]):
        Dcorr_RT_fo.append(np.corrcoef(food,DRT)[0][1])
    if not m.isnan(np.corrcoef(food,SRT)[0][1]):
        Scorr_RT_fo.append(np.corrcoef(food,SRT)[0][1])
    if not m.isnan(np.corrcoef(food,NORT)[0][1]):
        NOcorr_RT_fo.append(np.corrcoef(food,NORT)[0][1])
    # morning
    subset_mor = mdf[mdf['userID_x']==part]
    sleep = np.array(subset_mor['sleep'])
    if not m.isnan(np.corrcoef(sleep,subset_mor['OKDsubj_RT_oo'])[0][1]):
        Dcorr_RT_sl.append(np.corrcoef(sleep,subset_mor['OKDsubj_RT_oo'])[0][1])
    if not m.isnan(np.corrcoef(sleep,subset_mor['OKSsubj_RT_oo'])[0][1]):
        Scorr_RT_sl.append(np.corrcoef(sleep,subset_mor['OKSsubj_RT_oo'])[0][1])
    if not m.isnan(np.corrcoef(sleep,subset_mor['OKubj_RT_no'])[0][1]):
        NOcorr_RT_sl.append(np.corrcoef(sleep,subset_mor['OKubj_RT_no'])[0][1])
    
        
Dp_RT_mo = stats.ttest_1samp(Dcorr_RT_mo,0)[1]    
Dp_RT_fo = stats.ttest_1samp(Dcorr_RT_fo,0)[1]  
Dp_RT_sl = stats.ttest_1samp(Dcorr_RT_sl,0)[1]  
Dp_RT_st = stats.ttest_1samp(Dcorr_RT_st,0)[1]   

Sp_RT_mo = stats.ttest_1samp(Scorr_RT_mo,0)[1]    
Sp_RT_fo = stats.ttest_1samp(Scorr_RT_fo,0)[1]  
Sp_RT_sl = stats.ttest_1samp(Scorr_RT_sl,0)[1]  
Sp_RT_st = stats.ttest_1samp(Scorr_RT_st,0)[1]  

NOp_RT_mo = stats.ttest_1samp(NOcorr_RT_mo,0)[1]    
NOp_RT_fo = stats.ttest_1samp(NOcorr_RT_fo,0)[1]  
NOp_RT_sl = stats.ttest_1samp(NOcorr_RT_sl,0)[1]  
NOp_RT_st = stats.ttest_1samp(NOcorr_RT_st,0)[1] 

ptable = [['',' RT in DO',' RT in SO', 'RT in NO'],
          ['mood',roundP(Dp_RT_mo),roundP(Sp_RT_mo),roundP(NOp_RT_mo)],
         ['food',roundP(Dp_RT_fo),roundP(Sp_RT_fo),roundP(NOp_RT_fo)],
         ['stress',roundP(Dp_RT_st),roundP(Sp_RT_st),roundP(NOp_RT_st)],
         ['sleep',roundP(Dp_RT_sl),roundP(Sp_RT_sl),roundP(NOp_RT_sl)]]

#display(Latex(r"\newpage"))

display(HTML(tabulate.tabulate(ptable, tablefmt='html')))
#print(tabulate.tabulate(ptable))
#pt.PrettyTable(ptable)

In [ ]:
print([np.nanmean(NOcorr_RT_mo),np.nanmean(NOcorr_RT_st),np.nanmean(NOcorr_RT_sl),np.nanmean(NOcorr_RT_fo)])

In [ ]:
print([np.nanmean(NOp_RT_mo),np.nanmean(NOp_RT_st),np.nanmean(NOp_RT_sl),np.nanmean(NOp_RT_fo)])

## Correlation between psychometric variables

In [ ]:
Dcorr_acc_oo,Dcorr_oo_rt,Dcorr_oo_stsigma,Dcorr_oo_nosigma = [[] for _ in range(4)]
Scorr_acc_oo,Scorr_oo_rt,Scorr_oo_stsigma,Scorr_oo_nosigma = [[] for _ in range(4)]
DOSO_corr,corr_acc_rt,corr_acc_stsigma,corr_acc_nosigma,corr_stsigma_nosigma\
= [[] for _ in range(5)]
ind = -1
for part in userids:
    ind += 1
    subset = adf[adf['userID_x']==part]
    NOperf = np.array(subset['subj_perf_no'])
    NORT = np.array(subset['OKubj_RT_no'])
    DO = np.array(subset['Dsubj_optout_oo'])
    SO = np.array(subset['Ssubj_optout_oo'])
    st_sigma = np.array(subset['stair_sigma'])
    no_sigma = np.array(subset['no_sigma'])
    # delete nan values
    NOperf = NOperf[~np.isnan(NOperf)]
    DO = DO[~np.isnan(DO)]
    SO = SO[~np.isnan(SO)]
    NORT = NORT[~np.isnan(NORT)]  
    st_sigma = st_sigma[~np.isnan(st_sigma)]
    no_sigma = no_sigma[~np.isnan(no_sigma)]
    
    Dcorr_acc_oo.append(np.corrcoef(NOperf,DO)[0][1])
    corr_acc_rt.append(np.corrcoef(NOperf,NORT)[0][1])
    Dcorr_oo_rt.append(np.corrcoef(DO,NORT)[0][1])
    DOSO_corr.append(np.corrcoef(DO,SO)[0][1])
    Scorr_acc_oo.append(np.corrcoef(NOperf,SO)[0][1])
    Scorr_oo_rt.append(np.corrcoef(SO,NORT)[0][1])
    Dcorr_oo_stsigma.append(np.corrcoef(DO,st_sigma)[0][1])
    Scorr_oo_stsigma.append(np.corrcoef(SO,st_sigma)[0][1])
    Dcorr_oo_nosigma.append(np.corrcoef(DO,no_sigma)[0][1])
    Scorr_oo_nosigma.append(np.corrcoef(SO,no_sigma)[0][1])
    corr_acc_stsigma.append(np.corrcoef(NOperf,st_sigma)[0][1])
    corr_acc_nosigma.append(np.corrcoef(NOperf,no_sigma)[0][1])
    corr_stsigma_nosigma.append(np.corrcoef(st_sigma,no_sigma)[0][1])

Dp_acc_oo = stats.ttest_1samp(Dcorr_acc_oo,0)[1]     
Dp_oo_rt = stats.ttest_1samp(Dcorr_oo_rt,0)[1]
Dp_oo_stsigma = stats.ttest_1samp(Dcorr_oo_stsigma,0)[1]
Dp_oo_nosigma = stats.ttest_1samp(Dcorr_oo_nosigma,0)[1]

Sp_acc_oo = stats.ttest_1samp(Scorr_acc_oo,0)[1]      
Sp_oo_rt = stats.ttest_1samp(Scorr_oo_rt,0)[1]  
Sp_oo_stsigma = stats.ttest_1samp(Scorr_oo_stsigma,0)[1]
Sp_oo_nosigma = stats.ttest_1samp(Scorr_oo_nosigma,0)[1]

p_acc_rt = stats.ttest_1samp(corr_acc_rt,0)[1]   
DOSO_p = stats.ttest_1samp(DOSO_corr,0)[1]  
p_acc_stsigma = stats.ttest_1samp(corr_acc_stsigma,0)[1]   
p_acc_nosigma = stats.ttest_1samp(corr_acc_nosigma,0)[1]   
p_stsigma_nosigma = stats.ttest_1samp(corr_stsigma_nosigma,0)[1]   

ptable = [['','NO Perf','DO','NO RT',' SO','St sigma','NO sigma'],
          ['NO Perf',0,roundP(Dp_acc_oo),roundP(p_acc_rt),roundP(Sp_acc_oo),\
          roundP(p_acc_stsigma),roundP(p_acc_nosigma)],
          ['DO',roundP(Dp_acc_oo),0,roundP(Dp_oo_rt),roundP(DOSO_p),\
          roundP(Dp_oo_stsigma),roundP(Dp_oo_nosigma)],
          ['NO RT',roundP(p_acc_rt),roundP(Dp_oo_rt),0,roundP(Sp_oo_rt),\
          '--','--'],
          ['SO',roundP(Sp_acc_oo),roundP(DOSO_p),roundP(Sp_oo_rt),0,\
          roundP(Sp_oo_stsigma), roundP(Sp_oo_nosigma)],
         ['St sigma',roundP(p_acc_stsigma),roundP(Dp_oo_stsigma),'--',\
          roundP(Sp_oo_stsigma),0,roundP(p_stsigma_nosigma)],
         ['NO sigma',roundP(p_acc_nosigma),roundP(Dp_oo_nosigma),'--',\
          roundP(Sp_oo_nosigma),roundP(p_stsigma_nosigma),0]]

display(HTML(tabulate.tabulate(ptable, tablefmt='html'))) 

In [ ]:
corr_matrix =[[1,np.nanmean(Dcorr_acc_oo),np.nanmean(Scorr_acc_oo),np.nanmean(corr_acc_rt)],
          [np.nanmean(Dcorr_acc_oo),1,np.nanmean(DOSO_corr),np.nanmean(Dcorr_oo_rt)],
          [np.nanmean(Scorr_acc_oo),np.nanmean(DOSO_corr),1,np.nanmean(Scorr_oo_rt)],
          [np.nanmean(corr_acc_rt),np.nanmean(Dcorr_oo_rt),np.nanmean(Scorr_oo_rt),1]]
print(corr_matrix)

In [ ]:
pmatrix = [[0,Dp_acc_oo,Sp_acc_oo,p_acc_rt],
          [Dp_acc_oo,0,DOSO_p,Dp_oo_rt],
          [Sp_acc_oo,DOSO_p,0,Sp_oo_rt],
          [p_acc_rt,Dp_oo_rt,Sp_oo_rt,0]]

print(pmatrix)

In [ ]:
r_subjNOS_so_do,r_subjSIG_so_do = [[] for _ in range(2)]
ind = -1
for part in userids:
    ind += 1
    subset = adf[adf['userID_x']==part]
    DO = np.array(subset['Dsubj_optout_oo'])
    SO = np.array(subset['Ssubj_optout_oo'])
    
    # D optout & S optout
    slope_so, intercept_so, r_value_so, p_value_so, std_err_so = stats.linregress(SO,DO)
    if p_value_so<0.05:
        r_subjSIG_so_do.append(r_value_so)
    else:
        r_subjNOS_so_do.append(r_value_so)

In [ ]:
# write the result in file
dict_ = {
    "r_subjSIG_so_do":list(r_subjSIG_so_do),
    "r_subjNOS_so_do":list(r_subjNOS_so_do)
}
# Serializing json  
json_object = json.dumps(dict_) 

# Writing to sample.json 
with open('/home/cbclab/Dropbox/postdocUPF/jatos_experiments/gabor/notebooks/results_tanda_1/corr_Doptout_Soptout.json', "w") as outfile: 
    outfile.write(json_object) 

In [ ]:
fig, ax = plt.subplots(2,2,figsize=(8,6))
plt.subplots_adjust(hspace = 0.4) 
plt.subplots_adjust(wspace = 0.4)
ax[0,0].hist(corr_acc_stsigma,color='g')
ax[0,0].text(0.1,1.3,'ttest p: '+str(roundP(p_acc_stsigma)), ha='left', wrap=True,fontsize=18)
ax[0,0].text(0.1,2.3,'mean r: '+str(roundP(np.nanmean(corr_acc_stsigma))), ha='left', wrap=True,fontsize=18)
ax[0,0].set_title("NO acc & staircase noise")
ax[0,0].set_ylabel('Counts')
ax[0,0].set_xlim(-1.1,1.1)
ax[0,0].axvline(0,color='k')

ax[0,1].hist(Dcorr_oo_stsigma,color='r')
ax[0,1].text(0.1,2.5,'ttest p: '+str(roundP(Dp_oo_stsigma)), ha='left', wrap=True,fontsize=18)
ax[0,1].text(0.1,1.5,'mean r: '+str(roundP(np.nanmean(Dcorr_oo_stsigma))), ha='left', wrap=True,fontsize=18)
ax[0,1].set_title("DO & staircase noise")
ax[0,1].set_xlim(-1.1,1.1)
ax[0,1].axvline(0,color='k')

ax[1,0].hist(corr_acc_nosigma,color='g')
ax[1,0].text(0.1,1.3,'ttest p: '+str(roundP(p_acc_nosigma)), ha='left', wrap=True,fontsize=18)
ax[1,0].text(0.1,2.3,'mean r: '+str(roundP(np.nanmean(corr_acc_nosigma))), ha='left', wrap=True,fontsize=18)
ax[1,0].set_title("NO acc & NO noise")
ax[1,0].set_ylabel('Counts')
ax[1,0].set_xlim(-1.1,1.1)
ax[1,0].axvline(0,color='k')
ax[1,0].set_xlabel('Pearson corr coeff')

ax[1,1].hist(Dcorr_oo_nosigma,color='r')
ax[1,1].text(0.1,3.5,'ttest p: '+str(roundP(Dp_oo_nosigma)), ha='left', wrap=True,fontsize=18)
ax[1,1].text(0.1,2.5,'mean r: '+str(roundP(np.nanmean(Dcorr_oo_nosigma))), ha='left', wrap=True,fontsize=18)
ax[1,1].set_title("DO & NO noise")
ax[1,1].set_xlim(-1.1,1.1)
ax[1,1].axvline(0,color='k')
ax[1,1].set_xlabel('Pearson corr coeff')

plt.show()

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(18,3))
plt.subplots_adjust(hspace = 0.4) 
plt.subplots_adjust(wspace = 0.4) 
ax[0].hist(Dcorr_acc_oo,color='r')
ax[0].text(0.1,1.3,'ttest p: '+str(roundP(Dp_acc_oo)), ha='left', wrap=True,fontsize=18)
ax[0].text(0.1,2.3,'mean r: '+str(roundP(np.nanmean(Dcorr_acc_oo))), ha='left', wrap=True,fontsize=18)
ax[0].set_title("NO acc & oo DO")
ax[0].set_xlabel('Pearson corr coeff')
ax[0].set_ylabel('Counts')
ax[0].set_xlim(-1.1,1.1)
ax[0].axvline(0,color='k')

ax[1].hist(Scorr_acc_oo,color='b')
ax[1].text(0.1,3.5,'ttest p: '+str(roundP(Sp_acc_oo)), ha='left', wrap=True,fontsize=18)
ax[1].text(0.1,2.5,'mean r: '+str(roundP(np.nanmean(Scorr_acc_oo))), ha='left', wrap=True,fontsize=18)
ax[1].set_title("NO acc & oo SO")
ax[1].set_xlabel('Pearson corr coeff')
ax[1].set_xlim(-1.1,1.1)
ax[1].axvline(0,color='k')

ax[2].hist(DOSO_corr,color='m')
ax[2].text(0.1,3.5,'ttest p: '+str(roundP(DOSO_p)), ha='left', wrap=True,fontsize=18)
ax[2].text(0.1,2.5,'mean r: '+str(roundP(np.nanmean(DOSO_corr))), ha='left', wrap=True,fontsize=18)
ax[2].set_title("DO & SO")
ax[2].set_xlabel('Pearson corr coeff')
ax[2].set_xlim(-1.1,1.1)
ax[2].axvline(0,color='k')

plt.show()

## Overconfidence

Defined as $\frac{|H_L^{opt}-H_R^{opt}| - |H_L-H_R|}{|H_L^{opt}-H_R^{opt}|+|H_L-H_R|}$, we study the correlation between overconfidence and self-reports. We calculate $H_R^{opt}$ for the optout trials, defined as the stimulus value where $p(x>0|\hat{x})=2/3$. The optimal bias for leftward response corresponds to the stimulus value where $p(x<0|\hat{x})=1/3$: $H_L^{opt}=-H_R^{opt}$.

In [ ]:
adf['overConf'] = (np.abs(2*adf['HRopt_'])-np.abs(adf['DHL_oo']-adf['DHR_oo']))/(np.abs(2*adf['HRopt_'])+np.abs(adf['DHL_oo']-adf['DHR_oo']))

In [ ]:
adf.head()

In [ ]:
Dcorr_ov_mo,Dcorr_ov_fo,Dcorr_ov_sl,Dcorr_ov_st = [[] for _ in range(4)]
ind = -1
for part in userids:
    ind += 1
    subset = adf[adf['userID_x']==part]
    mood = np.array(subset['mood'])
    stress = np.array(subset['real_stress'])
    food = np.array(subset['food'])
    OV = np.array(subset['overConf'])
    if not m.isnan(np.corrcoef(mood,OV)[0][1]):
        Dcorr_ov_mo.append(np.corrcoef(mood,OV)[0][1])
    if not m.isnan(np.corrcoef(stress,OV)[0][1]):
        Dcorr_ov_st.append(np.corrcoef(stress,OV)[0][1])
    if not m.isnan(np.corrcoef(food,OV)[0][1]):
        Dcorr_ov_fo.append(np.corrcoef(food,OV)[0][1])
    # morning
    subset_mor = adf[(adf['userID_x']==part) & (adf['sessionID_x']%2==1)]
    sleep = np.array(subset_mor['sleep'])
    OVmor = subset_mor['overConf']
    if not m.isnan(np.corrcoef(sleep,OVmor)[0][1]):
        Dcorr_ov_sl.append(np.corrcoef(sleep,OVmor)[0][1])

Dp_ov_mo = stats.ttest_1samp(Dcorr_ov_mo,0)[1]    
Dp_ov_fo = stats.ttest_1samp(Dcorr_ov_fo,0)[1]  
Dp_ov_sl = stats.ttest_1samp(Dcorr_ov_sl,0)[1]  
Dp_ov_st = stats.ttest_1samp(Dcorr_ov_st,0)[1]     

ptable = [['','Overconfidence'],
          ['mood',np.round(Dp_ov_mo,2)],
         ['food',np.round(Dp_ov_fo,2)],
         ['stress',np.round(Dp_ov_st,2)],
         ['sleep',np.round(Dp_ov_sl,2)]]

#display(Latex(r"\newpage"))

display(HTML(tabulate.tabulate(ptable, tablefmt='html')))

## Risk aversion

We define the risk aversion as the difference between the choice opf opt-out option in the *DO* stage and the same amount in the *SO*.

In [ ]:
adf['risk_av'] = adf['Dsubj_perf_oo']-adf['Ssubj_perf_oo']

In [ ]:
Dcorr_ra_mo,Dcorr_ra_fo,Dcorr_ra_sl,Dcorr_ra_st = [[] for _ in range(4)]
ind = -1
for part in userids:
    ind += 1
    subset = adf[adf['userID_x']==part]
    mood = np.array(subset['mood'])
    stress = np.array(subset['real_stress'])
    food = np.array(subset['food'])
    RA = np.array(subset['risk_av'])
    if not m.isnan(np.corrcoef(mood,RA)[0][1]):
        Dcorr_ra_mo.append(np.corrcoef(mood,RA)[0][1])
    if not m.isnan(np.corrcoef(stress,RA)[0][1]):
        Dcorr_ra_st.append(np.corrcoef(stress,RA)[0][1])
    if not m.isnan(np.corrcoef(food,RA)[0][1]):
        Dcorr_ra_fo.append(np.corrcoef(food,RA)[0][1])
    # morning
    subset_mor = adf[(adf['userID_x']==part) & (adf['sessionID_x']%2==1)]
    sleep = np.array(subset_mor['sleep'])
    RAmor = subset_mor['risk_av']
    if not m.isnan(np.corrcoef(sleep,RAmor)[0][1]):
        Dcorr_ra_sl.append(np.corrcoef(sleep,RAmor)[0][1])

Dp_ra_mo = stats.ttest_1samp(Dcorr_ra_mo,0)[1]    
Dp_ra_fo = stats.ttest_1samp(Dcorr_ra_fo,0)[1]  
Dp_ra_sl = stats.ttest_1samp(Dcorr_ra_sl,0)[1]  
Dp_ra_st = stats.ttest_1samp(Dcorr_ra_st,0)[1]     

ptable = [['','Risk Aversion'],
          ['mood',np.round(Dp_ra_mo,2)],
         ['food',np.round(Dp_ra_fo,2)],
         ['stress',np.round(Dp_ra_st,2)],
         ['sleep',np.round(Dp_ra_sl,2)]]

#display(Latex(r"\newpage"))

display(HTML(tabulate.tabulate(ptable, tablefmt='html')))